## Process and Implementation
As an accompaniment to the videos we will follow the particle filter algorithm process and implementation details.
### Particle Filter Algorithm Steps and Inputs
The flowchart below represents the steps of the particle filter algorithm as well as its inputs.
![](https://video.udacity-data.com/topher/2017/August/5989f54e_02-l-pseudocode.00-00-47-13.still006/02-l-pseudocode.00-00-47-13.still006.png)

### Psuedo Code
This is an outline of steps you will need to take with your code in order to implement a particle filter for localizing an autonomous vehicle. The pseudo code steps correspond to the steps in the algorithm flow chart, initialization, prediction, particle weight updates, and resampling. Python implementation of these steps was covered in the previous lesson.

![](https://video.udacity-data.com/topher/2017/August/5989f6fb_02-l-pseudocode.00-00-14-28.still001/02-l-pseudocode.00-00-14-28.still001.png)

At the initialization step we estimate our position from GPS input. The subsequent steps in the process will refine this estimate to localize our vehicle.

![](https://video.udacity-data.com/topher/2017/August/5989f70c_02-l-pseudocode.00-00-16-01.still002/02-l-pseudocode.00-00-16-01.still002.png)

During the prediction step we add the control input (yaw rate & velocity) for all particles

![](https://video.udacity-data.com/topher/2017/August/5989f719_02-l-pseudocode.00-00-30-05.still003/02-l-pseudocode.00-00-30-05.still003.png)

During the update step, we update our particle weights using map landmark positions and feature measurements.

![](https://video.udacity-data.com/topher/2017/August/5989f726_02-l-pseudocode.00-00-35-08.still004/02-l-pseudocode.00-00-35-08.still004.png)

During resampling we will resample M times (M is range of 0 to length_of_particleArray) drawing a particle i (i is the particle index) proportional to its weight . Sebastian covered one implementation of this in his discussion and implementation of a resampling wheel.

![](https://video.udacity-data.com/topher/2017/August/5989f736_02-l-pseudocode.00-00-40-01.still005/02-l-pseudocode.00-00-40-01.still005.png)

The new set of particles represents the Bayes filter posterior probability. We now have a refined estimate of the vehicles position based on input evidence.

### Initialization

![](https://video.udacity-data.com/topher/2017/August/598a0660_03-l-initialization.00-01-53-01.still001/03-l-initialization.00-01-53-01.still001.png)

The most practical way to initialize our particles and generate real time output, is to make an initial estimate using GPS input. As with all sensor based operations, this step is impacted by noise.

#### Project Implementation
- Particles shall be implemented by sampling a Gaussian distribution, taking into account Gaussian sensor noise around the initial GPS position and heading estimates.
- Use the C++ standard library normal distribution and C++ standard library random engine functions to sample positions around GPS measurements.

```cpp
/**
 * print_samples_sol.cpp
 *
 * SOLUTION CODE
 * 
 * Print out to the terminal 3 samples from a normal distribution with
 * mean equal to the GPS position and IMU heading measurements and
 * standard deviation of 2 m for the x and y position and 0.05 radians
 * for the heading of the car. 
 *
 * Author: Tiffany Huang
 */

#include <iostream>
#include <random> // Need this for sampling from distributions

using std::normal_distribution;

/**
 * Prints samples of x, y and theta from a normal distribution
 * @param gps_x   GPS provided x position
 * @param gps_y   GPS provided y position
 * @param theta   GPS provided yaw
 */
void printSamples(double gps_x, double gps_y, double theta);


int main() {
  
  // Set GPS provided state of the car.
  double gps_x = 4983;
  double gps_y = 5029;
  double theta = 1.201;
  
  // Sample from the GPS provided position.
  printSamples(gps_x, gps_y, theta);
  
  return 0;
}


void printSamples(double gps_x, double gps_y, double theta) {
  std::default_random_engine gen;
  double std_x, std_y, std_theta;  // Standard deviations for x, y, and theta

  // TODO: Set standard deviations for x, y, and theta
  std_x = 2;
  std_y = 2;
  std_theta = 0.05; 

  // This line creates a normal (Gaussian) distribution for x
  normal_distribution<double> dist_x(gps_x, std_x);
  
  // TODO: Create normal distributions for y and theta
  normal_distribution<double> dist_y(gps_y, std_y);
  normal_distribution<double> dist_theta(theta, std_theta);

  for (int i = 0; i < 3; ++i) {
    double sample_x, sample_y, sample_theta;
    
    // TODO: Sample from these normal distributions like this: 
    //   sample_x = dist_x(gen);
    //   where "gen" is the random engine initialized earlier.
    sample_x = dist_x(gen);
    sample_y = dist_y(gen);
    sample_theta = dist_theta(gen);   
     
    // Print your samples to the terminal.
    std::cout << "Sample " << i + 1 << " " << sample_x << " " << sample_y << " " 
              << sample_theta << std::endl;
  }

  return;
}
```
### Prediction
![](https://video.udacity-data.com/topher/2017/August/598a0d55_05-l-predictionstep.00-00-38-28.still001/05-l-predictionstep.00-00-38-28.still001.png)
Now that we have initialized our particles it's time to predict the vehicle's position. Here we will use what we learned in the motion models lesson to predict where the vehicle will be at the next time step, by updating based on yaw rate and velocity, while accounting for Gaussian sensor noise.

![](./img/26.png)

### Update Step
!(https://www.youtube.com/watch?v=1Uq2QZKz3aI)

Note that the x and y errors are depicted from the point of view of the map (x is horizontal, y is vertical) rather than the point of view of the car where x is in the direction of the car’s heading,( i.e. It points to where the car is facing), and y is orthogonal (90 degrees) to the left of the x-axis (pointing out of the left side of the car).
![](https://video.udacity-data.com/topher/2017/August/598a1664_07-l-data-association-nearest-neighbor.00-00-17-03.still003/07-l-data-association-nearest-neighbor.00-00-17-03.still003.png)
![](https://video.udacity-data.com/topher/2017/August/598a167a_09-l-update-step.00-00-17-03.still001/09-l-update-step.00-00-17-03.still001.png)
Now that we have incorporated velocity and yaw rate measurement inputs into our filter, we must update particle weights based on LIDAR and RADAR readings of landmarks. We will practice calculating particle weights, later in this lesson, with the Particle Weights Quiz.

### Error
![](./img/27.png)

### Transformations and Associations
In the project you will need to correctly perform observation measurement transformations, along with identifying measurement landmark associations in order to correctly calculate each particle's weight. Remember, our ultimate goal is to find a weight parameter for each particle that represents how well that particle fits to being in the same location as the actual car.

In the quizzes that follow we will be given a single particle with its position and heading along with the car's observation measurements. We will first need to transform the car's measurements from its local car coordinate system to the map's coordinate system. Next, each measurement will need to be associated with a landmark identifier, for this part we will take the closest landmark to each transformed observation. Finally, we will use this information to calculate the weight value of the particle.
![](https://video.udacity-data.com/topher/2017/August/598b467e_localization-map-concept-copy/localization-map-concept-copy.png)
In the graph above we have a car (ground truth position) that observes three nearby landmarks, each one labeled OBS1, OBS2, OBS3. Each observation measurement has x, and y values in the car's coordinate system. We have a particle "P" (estimated position of the car) above with position (4,5) on the map with heading -90 degrees. The first task is to transform each observation marker from the vehicle's coordinates to the map's coordinates, with respect to our particle.

### Converting Landmark Observations

![](./img/28.png)
Here is another example that might help your intuition.

Referring to the figures below:

Suppose the map coordinate system (grey lines) and the vehicle coordinate system (orange lines) are offset, as depicted below. If we know the location of the observation in vehicle coordinates (grey lines), we would need to rotate the entire system, observation included, -45 degrees to find it in map coordinates (grey lines), Once this rotation is done, we can easily see the location of the observation in map coordinates.
#### Particle (blue dot) in Map Frame (grey)
![](https://video.udacity-data.com/topher/2017/October/59d7d8a1_45deg-1/45deg-1.png)
#### Particle (blue dot) in Vehicle Frame (orange)
![](https://video.udacity-data.com/topher/2017/October/59d7d8d9_45deg-2/45deg-2.png)

### Resources, Hints, and Tips
Without implementation of localization methods the car does not know where it is within an acceptable level of precision. The car knows particle coordinates and observation coordinates. The objective is to use the particle coordinates and heading to transform the car's frame of reference to the map's frame of reference, associate the observations, then use the associated observations in the map domain to update the particle weight.
Since we know the coordinates of the particle from the car's frame of reference we can use this information and a matrix rotation/translation to transform each observation from the car frame of reference to the map frame of reference. The particle is at (4,5) in the map coordinate system with a heading of -90 degrees. The figure indicates the heading by depicting the particle x-axis as pointing down (blue arrow). This is critical to understanding the matrix transformation we are about to perform.
By convention we define the car coordinate system with x pointing up and y rotated from x by pi/2 (+90 degrees). This is another way of saying that y is perpendicular and to the left of x. When x is pointing down, we have the mirror of this, with y perpendicular to the right. To visualize this make an L with your left index finger and thumb with palm facing away from you, this is our map frame of reference. Point the thumb towards the ceiling, this is the car coordinate convention, now point your thumb down, this is the orientation of the particle at (4,5).
Now consider the map frame of reference (make an L with your left index finger and thumb as above), this is a typical presentation of Cartesian coordinates, with x pointing right and y perpendicular to the left, pointing up. If we rotate our thumb down we have the particle orientation. To get back to the map orientation we must rotate counterclockwise by 90 degrees (thumb from pointing down back to pointing right). Try this a few times with your left hand. Notice that particle to map is a counterclockwise rotation (+90 degrees) nd map to particle is a clockwise rotation (-90 degrees).
The most straight forward way of rotating and translating coordinates is through homogenous transformation matrix (see below) using the angle of rotation required to get to the particle’s frame from the map’s point of view, -90 degrees. This way we can use theta directly. The alternative, which we will not cover here is to is to use -theta and a transformation matrix from the particle frame to the map frame.
This video is a great resource for developing a deeper understanding of how to solve this transformation problem - it covers the rotation transformation, and from there you just need to perform a translation.
Observations in the car coordinate system can be transformed into map coordinates (\text{x}_mx 
m
​	  and \text{y}_my 
m
​	 ) by passing car observation coordinates (\text{x}_cx 
c
​	  and \text{y}_cy 
c
​	 ), map particle coordinates (\text{x}_px 
p
​	  and \text{y}_py 
p
​	 ), and our rotation angle (-90 degrees) through a homogenous transformation matrix. This homogenous transformation matrix, shown below, performs rotation and translation.

#### Associations
Now that observations have been transformed into the map's coordinate space, the next step is to associate each transformed observation with a land mark identifier. In the map exercise above we have 5 total landmarks each identified as L1, L2, L3, L4, L5, and each with a known map location. We need to associate each transformed observation TOBS1, TOBS2, TOBS3 with one of these 5 identifiers. To do this we must associate the closest landmark to each transformed observation.

As a reminder:

TOBS1 = (6,3), TOBS2 = (2,2) and TOBS3 = (0,5).

```cpp

#include <cmath>
#include <iostream>

int main() {
  // define coordinates and theta
  double x_part, y_part, x_obs, y_obs, theta;
  x_part = 4;
  y_part = 5;
  x_obs = 2;
  y_obs = 2;
  theta = -M_PI/2; // -90 degrees

  // transform to map x coordinate
  double x_map;
  x_map = x_part + (cos(theta) * x_obs) - (sin(theta) * y_obs);

  // transform to map y coordinate
  double y_map;
  y_map = y_part + (sin(theta) * x_obs) + (cos(theta) * y_obs);

  // (6,3)
  std::cout << int(round(x_map)) << ", " << int(round((y_map)) << std::endl;

  return 0;
}
```

![](./img/29.png)

                                                
```cpp
                                                #include <iostream>
#include "multiv_gauss.h"

int main() {
  // define inputs
  double sig_x, sig_y, x_obs, y_obs, mu_x, mu_y;
  // define outputs for observations
  double weight1, weight2, weight3;
  // final weight
  double final_weight;
    
  // OBS1 values
  sig_x = 0.3;
  sig_y = 0.3;
  x_obs = 6;
  y_obs = 3;
  mu_x = 5;
  mu_y = 3;
  // Calculate OBS1 weight
  weight1 = multiv_prob(sig_x, sig_y, x_obs, y_obs, mu_x, mu_y);
  // should be around 0.00683644777551 rounding to 6.84E-3
  std::cout << "Weight1: " << weight1 << std::endl;
    
  // OBS2 values
  sig_x = 0.3;
  sig_y = 0.3;
  x_obs = 2;
  y_obs = 2;
  mu_x = 2;
  mu_y = 1;
  // Calculate OBS2 weight
  weight2 = multiv_prob(sig_x, sig_y, x_obs, y_obs, mu_x, mu_y);
  // should be around 0.00683644777551 rounding to 6.84E-3
  std::cout << "Weight2: " << weight2 << std::endl;
    
  // OBS3 values
  sig_x = 0.3;
  sig_y = 0.3;
  x_obs = 0;
  y_obs = 5;
  mu_x = 2;
  mu_y = 1;
  // Calculate OBS3 weight
  weight3 = multiv_prob(sig_x, sig_y, x_obs, y_obs, mu_x, mu_y);
  // should be around 9.83184874151e-49 rounding to 9.83E-49
  std::cout << "Weight3: " << weight3 << std::endl;
    
  // Output final weight
  final_weight = weight1 * weight2 * weight3;
  // 4.60E-53
  std::cout << "Final weight: " << final_weight << std::endl;
    
  return 0;
}
```
                                             